#### Loading data 

In [10]:
import os
from dotenv import load_dotenv
import pandas as pd
from twarc import Twarc2, expansions
from tweet_processing import StreamTweetProcessor

In [14]:
load_dotenv()

data_dir="../stream-seeding/data" 
twarc_client = Twarc2(
    consumer_key=os.environ["consumer_key"], 
    consumer_secret=os.environ["consumer_secret"],
    access_token=os.environ["access_token"], 
    access_token_secret=os.environ["access_token_secret"]
)

tp = StreamTweetProcessor(twarc_client=twarc_client, data_dir=data_dir)

group_name = "longevity-pranab"
#group_name = "CA-Abundance-Economy"

df_following, df_tweets, df_ref_tweets = tp.load_stream_seed_data(group_name) 

In [19]:
[df_following, df_tweets, df_ref_tweets] = map(lambda x: x.dropna(axis=1), [df_following, df_tweets, df_ref_tweets])

df_tweets

,Unnamed: 0,id,conversation_id,author_id,created_at,text,lang,source,public_metrics.like_count,public_metrics.quote_count,...,author.public_metrics.listed_count,author.public_metrics.tweet_count,author.url,author.verified,__twarc.retrieved_at,__twarc.url,__twarc.version,tweet_type,tweet_link,created_at.hour
0,0,1546652638917275649,1546652638917275649,301427990,2022-07-12 00:28:19+00:00,I'd like to have fluency with basic game theor...,en,Twitter Web App,75,1,...,861,1463,https://t.co/NdybXcN3no,False,2022-07-19T19:56:32+00:00,https://api.twitter.com/2/users/301427990/twee...,2.11.0,standalone,https://twitter.com/LauraDeming/status/1546652...,2022-07-12 00:00:00+00:00
1,1,1546636515920322560,1546636515920322560,301427990,2022-07-11 23:24:15+00:00,I wish there was a Serious Eats but for bio pr...,en,Twitter Web App,20,0,...,861,1463,https://t.co/NdybXcN3no,False,2022-07-19T19:56:32+00:00,https://api.twitter.com/2/users/301427990/twee...,2.11.0,standalone,https://twitter.com/LauraDeming/status/1546636...,2022-07-11 23:00:00+00:00
2,2,1542617941480333314,1542615757460082688,301427990,2022-06-30 21:15:52+00:00,"Half of it reads like a modern Medium post, th...",en,Twitter Web App,10,0,...,861,1463,https://t.co/NdybXcN3no,False,2022-07-19T19:56:32+00:00,https://api.twitter.com/2/users/301427990/twee...,2.11.0,self-reply,https://twitter.com/LauraDeming/status/1542617...,2022-06-30 21:00:00+00:00
3,3,1542617483420368897,1542615757460082688,301427990,2022-06-30 21:14:03+00:00,Thanks @michael_nielsen for pointing out that ...,en,Twitter Web App,18,0,...,861,1463,https://t.co/NdybXcN3no,False,2022-07-19T19:56:32+00:00,https://api.twitter.com/2/users/301427990/twee...,2.11.0,"self-reply,mention",https://twitter.com/LauraDeming/status/1542617...,2022-06-30 21:00:00+00:00
4,4,1542615757460082688,1542615757460082688,301427990,2022-06-30 21:07:11+00:00,I wrote a personal post about how 28 feels dif...,en,Twitter Web App,140,0,...,861,1463,https://t.co/NdybXcN3no,False,2022-07-19T19:56:32+00:00,https://api.twitter.com/2/users/301427990/twee...,2.11.0,standalone,https://twitter.com/LauraDeming/status/1542615...,2022-06-30 21:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7938,0,1479851646397992964,1479843409456160768,4902221,2022-01-08 16:25:01+00:00,@kulesatony @kanjun My head 🙃 https://t.co/MbI...,en,TweetDeck,1,0,...,392,164097,https://t.co/8xDYgWSkz9,False,2022-07-19T20:13:45+00:00,https://api.twitter.com/2/users/4902221/tweets...,2.11.0,"qt,reply",https://twitter.com/ArtirKel/status/1479851646...,2022-01-08 16:00:00+00:00
7939,1,1479846350917287941,1479843409456160768,4902221,2022-01-08 16:03:58+00:00,@kulesatony @kanjun the lookup table strikes a...,en,Twitter for Android,1,0,...,392,164097,https://t.co/8xDYgWSkz9,False,2022-07-19T20:13:45+00:00,https://api.twitter.com/2/users/4902221/tweets...,2.11.0,"self-reply,mention",https://twitter.com/ArtirKel/status/1479846350...,2022-01-08 16:00:00+00:00
7940,2,1479844656397492226,1479843409456160768,4902221,2022-01-08 15:57:14+00:00,"@kulesatony And also (all from my last links, ...",en,Twitter for Android,2,0,...,392,164097,https://t.co/8xDYgWSkz9,False,2022-07-19T20:13:45+00:00,https://api.twitter.com/2/users/4902221/tweets...,2.11.0,"qt,self-reply,mention",https://twitter.com/ArtirKel/status/1479844656...,2022-01-08 15:00:00+00:00
7941,3,1479843944464740353,1479843409456160768,4902221,2022-01-08 15:54:25+00:00,@kulesatony Indeed https://t.co/l3key7jd0Y,en,Twitter for Android,3,0,...,392,164097,https://t.co/8xDYgWSkz9,False,2022-07-19T20:13:45+00:00,https://api.twitter.com/2/users/4902221/tweets...,2.11.0,reply,https://twitter.com/ArtirKel/status/1479843944...,2022-01-08 15:00:00+00:00


In [8]:
## Get a set of usernames that each seed account follows
users_following = {}
for user in df_following["referencer.username"].unique().tolist():
    users_following[user] = set(df_following[df_following["referencer.username"] == user]["username"].tolist())
    
## index will be useful to access metadata (followers_count) about these accounts
df_following.set_index("username", inplace=True)


### Create a dataframe that will show the following_overlap for each of the accounts followed by at least 1 of the seed accounts. 
### We are most interested in the intersection among the following of those accounts
### If the accounts are picked well, they will likely have interesting intersection among their shared interest

following_overlap = {}

for stream_user, following in users_following.items():
    for followed_user in following: 
        following_overlap.setdefault(followed_user, []).append(stream_user)

df_data = []

for followed_user, stream_users in following_overlap.items():
    num_followers_of_followed = df_following.loc[followed_user]["public_metrics.followers_count"]
    if isinstance(num_followers_of_followed, pd.Series):
        num_followers_of_followed = num_followers_of_followed.iloc[0]
    df_data.append([followed_user, stream_users, len(stream_users), num_followers_of_followed, f"https://twitter.com/{followed_user}"])
    
overlap_df = pd.DataFrame(df_data, columns=["followed.username", "stream_users", "num_stream_users", "num_followers_of_followed", "profile_link"])
overlap_df["num_stream_users"].value_counts()

1    7097
2     393
3      33
Name: num_stream_users, dtype: int64

### Show Overlap
Here, we show users that are followed by multiple seed accounts, sorted by the number of followers of that accounts. 

Filtering out to the lower follower accounts are a signal that that person talks abou their shared interest, and is an easy way to avoid constant reccomendation of accounts like Elon

In [9]:
NUM_SEED_ACCOUNTS = 3
overlap_df[overlap_df["num_stream_users"]==NUM_SEED_ACCOUNTS].sort_values("num_followers_of_followed")

,followed.username,stream_users,num_stream_users,num_followers_of_followed,profile_link
3006,gillibits,"[rklau, anniefryman, hanlonbt]",3,1380,https://twitter.com/gillibits
266,thuddwhirr,"[rklau, anniefryman, hanlonbt]",3,1790,https://twitter.com/thuddwhirr
2239,mishachellam,"[rklau, anniefryman, hanlonbt]",3,4328,https://twitter.com/mishachellam
2970,tchu88,"[rklau, anniefryman, hanlonbt]",3,8278,https://twitter.com/tchu88
1285,cbracy,"[rklau, anniefryman, hanlonbt]",3,8297,https://twitter.com/cbracy
4264,allafarce,"[rklau, anniefryman, hanlonbt]",3,8788,https://twitter.com/allafarce
2710,profjohnapowell,"[rklau, anniefryman, hanlonbt]",3,9165,https://twitter.com/profjohnapowell
2735,Lenny_Mendonca,"[rklau, anniefryman, hanlonbt]",3,10171,https://twitter.com/Lenny_Mendonca
616,NancySkinnerCA,"[rklau, anniefryman, hanlonbt]",3,12838,https://twitter.com/NancySkinnerCA
2197,starsandrobots,"[rklau, anniefryman, hanlonbt]",3,21575,https://twitter.com/starsandrobots
